<a href="https://colab.research.google.com/github/Flychuban/Pneumonia-Prediction/blob/main/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Dense, Input, Lambda, Flatten
from keras.models import Model, Sequential
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
from matplotlib import pyplot as plt

In [2]:
# Resize all images
images_size = [224, 224]

train_path = '/content/drive/MyDrive/PneumoniaPrediction/data/train'
test_path = '/content/drive/MyDrive/PneumoniaPrediction/data/test'

In [3]:
vgg = VGG19(input_shape=images_size + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [4]:
for layer in vgg.layers:
  layer.trainable = False

In [7]:
folders = glob(train_path + '/*')

In [8]:
folders

['/content/drive/MyDrive/PneumoniaPrediction/data/train/PNEUMONIA',
 '/content/drive/MyDrive/PneumoniaPrediction/data/train/NORMAL']

In [9]:
# Add new layers to the model
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(2, activation='softmax')(x)

In [12]:
# Making new model based on vgg19
model = Model(inputs=vgg.input, outputs=prediction)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# We are generating more training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [18]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical')

Found 624 images belonging to 2 classes.
